In [ ]:
import FinanceDataReader as fdr
import pandas as pd
from pykrx import stock
from datetime import datetime

# 1. 기간 설정
start_date_str = '20150101'
end_date_str = datetime.now().strftime('%Y%m%d')
start_date_fdr = '2015-01-01'
end_date_fdr = datetime.now().strftime('%Y-%m-%d')

try:
    # 2. 일별 데이터 수집
    usd_krw = fdr.DataReader('USD/KRW', start_date_fdr, end_date_fdr)['Close']
    kospi = fdr.DataReader('KS11', start_date_fdr, end_date_fdr)['Close']
    investors = stock.get_market_trading_value_by_date(start_date_str, end_date_str, "KOSPI")

    # 3. 데이터 통합 전처리
    investors.index = pd.to_datetime(investors.index)
    target_col = [col for col in investors.columns if '외국인' in col]
    
    if target_col:
        df_daily = pd.DataFrame({
            'USD_KRW': usd_krw,
            'KOSPI': kospi,
            'Foreign_Net_Buy': investors[target_col[0]]
        })

    df_daily.index = pd.to_datetime(df_daily.index)
    df_daily = df_daily.dropna()

    # 4. 월별 리샘플링 (월초 기준: 'MS')
    # MS = Month Start (각 달의 1일로 표시됨)
    df_monthly = pd.DataFrame()
    
    # 환율과 지수는 해당 월의 마지막 종가를 가져오되, 라벨만 월초로 표시
    df_monthly['USD_KRW'] = df_daily['USD_KRW'].resample('MS').last()
    df_monthly['KOSPI'] = df_daily['KOSPI'].resample('MS').last()
    
    # 외국인 순매수는 해당 월의 전체 합계
    df_monthly['Foreign_Net_Buy_Sum'] = df_daily['Foreign_Net_Buy'].resample('MS').sum()

    # 5. 결과 저장 및 출력
    file_name = 'foreigner_stock_trading(month).csv'
    df_monthly.to_csv(file_name, encoding='utf-8-sig')
    
    print(f"\n 월초 기준 데이터 파일이 생성되었습니다: '{file_name}'")
    print("-" * 40)
    print(df_monthly.head()) # 데이터 시작 부분 확인 (2015-01-01 등)

except Exception as e:
    print(f"\n 에러 발생: {e}")

데이터 수집 및 [월초] 기준 변환을 시작합니다...

✅ 월초 기준 데이터 파일이 생성되었습니다: 'foreigner_stock_trading(month_start).csv'
----------------------------------------
                USD_KRW    KOSPI  Foreign_Net_Buy_Sum
2015-01-01  1098.800049  1949.26        -1.044956e+12
2015-02-01  1100.670044  1985.80         1.327869e+12
2015-03-01  1107.099976  2041.03         2.864169e+12
2015-04-01  1075.060059  2127.17         3.471482e+12
2015-05-01  1107.000000  2110.89         6.587259e+11
